In [1]:
#0.创建数据表
#1.爬虫爬取上证股票数据
#2.连接数据库
#2.5.显示数据
#3.读取数据库
#4.更新数据库


#0.创建数据表-----------------------------------------------------------------------------

import pymysql

def create_table():
    if not connect_db() == False:
        conn, cur = connect_db()
        sql = """create table copredict_source_stock(
                 id int(10) auto_increment primary key,
                 date date,
                 stock varchar(64),
                 price float)"""
        cur.execute(sql)
    
        cur.execute('desc copredict_source_stock')
        print(cur.fetchall())

    
    
    
#1.爬虫爬取上证股票数据-----------------------------------------------------------------------------

import tushare as ts

def get_stock(): 
    pro = ts.pro_api('7d06168119150384780b531dd1ab3a9f45de57fde4688db05b19ea95')
    df = pro.daily(ts_code = '000001.SZ', start_date = '20190218', fields = ['trade_date', 'close'])
    return df

#2.连接数据库-----------------------------------------------------------------------------


import pymysql

def connect_db():
    try:
        #建立connect路径
        conn = pymysql.connect(host = 'sql.b51.vhostgo.com', db = 'passvisor', user = 'passvisor', password = 'Xudamin123', charset = 'utf8')
        #建立cursor
        cur = conn.cursor()  
        return conn, cur    #返回路径、cursor，供下文使用
    except:
        return False       #如上述代码请求不成功，返回False字样

#2.5.显示数据-----------------------------------------------------------------------------

import pymysql

def show_table_data(table_name):                #定义显示数据表函数
    if not connect_db == False:                #如果连接数据库没有不成功，
        sql = 'select * from %s'%(table_name)   #sql指令执行全选数据表内容
        conn, cur = connect_db()                #connect路径与cursor来自connect_db()
        cur.execute(sql)                        #执行sql指令
        print(cur.fetchall())                   #显示所有
        
#3.读取数据库-----------------------------------------------------------------------------


import pandas as pd
import pymysql

def read_table_data(table_name):
    if not connect_db() == False:                     #如果连接数据库没有不成功，
        conn, cur = connect_db()                       #connect路径与cursor来自connect_db()
        sql = 'select * from %s'%(table_name)         #sql指令执行全选数据表内容
        db_data = pd.read_sql(sql = sql, con = conn) #用pandas格式读取
    return db_data                                  #显示数据

#4.更新数据库-----------------------------------------------------------------------------

import pymysql
import datetime

def updated_data():                       
    if not connect_db == False:         #如果连接数据库没有不成功，
        conn, cur = connect_db()         #connect路径与cursor来自connect_db() 
        
        s_data = get_stock()  #定义s_data为最新爬取的数据
        db_data = read_table_data('copredict_source_stock')                                           #定义db_data已有数据
        k = 0                                                                                        #赋值k
        
        for i in range(len(s_data)):                                                                #建立for循环，开始更新数据  
            #如果最新爬取的数据记录里的日期没有存在于已有数据内时，
            uf_date = s_data['trade_date'][i]
            f_date = '{}{}{}{}{}'.format(uf_date[0:4], '-', uf_date[4:6], '-', uf_date[6:8])
    
            if datetime.datetime.strptime(f_date, '%Y-%m-%d').date() not in list(db_data['date']):  
                #写入该记录
                sql = "insert into copredict_source_stock(date, stock, price) value('%s', '%s', %s)"%(s_data['trade_date'][i], '000001.SZ', s_data['close'][i])
                cur.execute(sql)  #执行sql指令
                k = k + 1         #为k赋值，若有变化就+1
        conn.commit()
        print('Done! Update {} records'.format(k))   #显示结果


#-----------------------------------------------------------------------------



if __name__ == '__main__': 
    updated_data()     #从入口程序运行程序
   
    #read_table_data('copredict_source_stock')
    
    #show_table_data('copredict_source_stock')

    #get_stock()
    
    #create_table()

Done! Update 3 records
